In [ ]:
# main
import pandas as pd
import matplotlib.pyplot as plt

filename = './startup+account+analysis+1028.xlsx'
df = pd.read_excel(filename, sheet_name='1028', header=[1])

df.shape
#df.info


In [ ]:
# remove charge
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

In [ ]:
"""
cols = []
cols_raw = df.columns.values.tolist()
for col in cols_raw:
    #print (col)
    if str(col).startswith('202'):
        colName = '-'.join( str(col).split('-')[0:2] )
        cols.append(str(colName))
    else:
        cols.append(col)

df.columns = cols

"""


In [ ]:
# cols to rows
cols = df.columns
dfMelt = pd.melt(df, id_vars=cols[0:7], value_vars=cols[8:])
dfMelt.shape

In [ ]:
#calc anomal values

left = pd.DataFrame(dfMelt.groupby('Master Account Id').median() - pd.DataFrame(dfMelt.groupby('Master Account Id').std()))
right = pd.DataFrame(dfMelt.groupby('Master Account Id').median() + pd.DataFrame(dfMelt.groupby('Master Account Id').std()))
left.columns = ['left']
right.columns = ['right']
dfMelt = dfMelt.merge(left, left_on='Master Account Id', right_index=True)
dfMelt = dfMelt.merge(right, left_on='Master Account Id', right_index=True)
dfMelt_down = dfMelt[dfMelt['value'] < dfMelt['left'] ]
dfMelt_up = dfMelt[dfMelt['value'] > dfMelt['right']]
#dfMelt = dfMelt.drop(['left', 'right'], axis=1)
#dfMelt

In [ ]:
dfMelt_down.to_excel('/home/ec2-user/SageMaker/kguo/down.xlsx')
dfMelt_down

In [ ]:
#change column
colsNew=['masterAccountId', 'masterAccountName', 'masterAccountOwner', 'phase', 'territory', 'at1', 'at2', 'variable', 'value', 'left', 'right']
dfMelt_up.columns=colsNew
dfMelt_down.columns=colsNew

In [ ]:
dfMelt_up.to_excel('/home/ec2-user/SageMaker/kguo/up.xlsx')
dfMelt_down.to_excel('/home/ec2-user/SageMaker/kguo/down.xlsx')

In [ ]:
## pre-deploy
dfMelt_up.to_json('/home/ec2-user/SageMaker/kguo/container/up.json', 'records')
dfMelt_down.to_json('/home/ec2-user/SageMaker/kguo/container/down.json', 'records')

In [ ]:
#deploy
!container/buildAndPushECR.sh

In [52]:
def addFollowUp(payload=None):
    headers = {
            "Content-Type": "application/json",
            "Access-Control-Allow-Headers": "Content-Type",
            "Access-Control-Allow-Origin": "*",
            "Access-Control-Allow-Methods": "OPTIONS,POST,GET"
        }
    
    payload.get('r').get('a')
    
    accountId = payload.get('r').get('accountId')
    accountName = payload.get('r').get('accountName')
    accountOwner = payload.get('r').get('accountOwner')
    month = payload.get('r').get('variable')
    rev = payload.get('r').get('value')
    abnormalType = payload.get('abnormalType')
    priority = payload.get('rgValue')
    followup = payload.get('textValue')
    
    ddb = boto3.resource('dynamodb')
    followups = followups(ddb)
    followups_exist = followups.exists(table_name)
    if not followups_exist:
        followups.create_table(table_name)
        
    LOGGER.info('here we go @1... ')
    
    fu = { 'accountId': accountId,
           'accountName': accountName,
           'accountOwner': accountOwner,
           'month': month,
           'rev': rev,
           'abnormalType': abnormalType,
           'priority': priority,
           'createTime': now() }
    
    LOGGER.info('here we go @2... ')
    
    followups.add_followup(fu)
    
    LOGGER.info('here we go @3... ')
            
    return {
        "statusCode": 200,
        "headers":headers,
        "body": json.dumps( {'code': 200, 'success': 'true', 'Data': [] }  )          
    }


str = '{"accountId":"abc"}'
payload = json.loads(str)
payload.get('accountId')

addFollowUp(payload=payload)


AttributeError: 'NoneType' object has no attribute 'get'

In [59]:
from datetime import datetime

fu = {  'createTime': round(datetime.now().timestamp()) }
fu

{'createTime': 1671179830}